# ETL Pipeline Preparation

## 1. Import libraries and load datasets.
- Import Python libraries
- Load two files `data/messages.csv` and `data/categories.csv` into dataframes.

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
%config Completer.use_jedi = False

In [3]:
messages = pd.read_csv('data/messages.csv')
categories = pd.read_csv('data/categories.csv')

In [4]:
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [5]:
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


## 2. Merge datasets

In [6]:
# Number of matching id = 26428, which equals the total number of records
(messages.id == categories.id).sum()

26248

In [7]:
df = messages.merge(categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [8]:
df.isnull().sum()

id                0
message           0
original      16140
genre             0
categories        0
dtype: int64

## 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. 
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [10]:
# Create a category list using the first row of categories dataframe
category_list = categories.loc[0, 'categories'].split(';')

# Remove boolean tags i.e. '-1'
category_list = [category.split('-')[0] for category in category_list]

# Printing the number of different categories
len(category_list) # 36 different categories

36

In [14]:
# Create a dataframe of expanded columns of 36 different categories
categories_expanded = df['categories'].str.split(';', expand=True)
categories_expanded.columns = category_list

In [15]:
categories_expanded.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


## 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.

In [16]:
for column in categories_expanded:
    # set each value to be the last character of the string
    categories_expanded[column] = categories_expanded[column].str.split('-').str[1]
    
    # convert column from string to numeric
    categories_expanded[column] = categories_expanded[column].astype(int)

categories_expanded.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Checking the converted data

In [17]:
# Inspecting the binary transformed data
categories_expanded.describe()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26386.000000,26386.000000,26386.000000,26386.000000,26386.00000,26386.000000,26386.000000,26386.00000,26386.000000,26386.0,...,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000,26386.000000
mean,0.775032,0.171038,0.004586,0.415144,0.07955,0.049989,0.027477,0.01785,0.032707,0.0,...,0.011711,0.043773,0.278292,0.082506,0.093383,0.010687,0.093269,0.020200,0.052263,0.193777
std,0.435692,0.376549,0.067564,0.492756,0.27060,0.217926,0.163471,0.13241,0.177871,0.0,...,0.107583,0.204594,0.448166,0.275139,0.290974,0.102828,0.290815,0.140687,0.222560,0.395264
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
# Check for columns that don't contain both 0, 1 binary values
for col in categories_expanded : 
    if (categories_expanded[col].max() != 1) or (categories_expanded[col].min() != 0) :
        print(col, categories_expanded[col].unique())

related [1 0 2]
child_alone [0]


In [19]:
categories_expanded.related.value_counts()

1    20042
0     6140
2      204
Name: related, dtype: int64

In [20]:
categories_expanded.child_alone.value_counts()

0    26386
Name: child_alone, dtype: int64

It all looks good but `related` column has **value of 2**. After a little bit of research, `related` column does have value of 2, which indicates "Indirectly Related." There are 193 records that corresponds to 'indirecly related' which is less than 1% of the data. It is adviced that it can be converted to `1` (mode value).

Source of the advice: a Udacity's knowledge post https://knowledge.udacity.com/questions/136791

Also `child_alone` column does not really have any value fields other than 0, but will be kept for the purpose of data consistency.

In [21]:
# Replace integer 2 to 1
categories_expanded['related'] = categories_expanded['related'].apply(lambda x: 1 if x == 2 else x)

## 5. Replace `categories` column in `df` with new category columns.

In [22]:
categories_expanded.isnull()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26381,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26382,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26383,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26384,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
# Concetenate new categories columns
df_concat = pd.concat([df, categories_expanded], axis=1)

# Drop the 'categories' column that is no longer necessary
df_concat = df_concat.drop(columns='categories')

In [26]:
# Confirm the number of features 
# Should be 40 == 5 original + 36 expanded - 1 duplicated column
assert df_concat.shape[1] == 40

## 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [27]:
# Number of records for original dataset
df_concat.shape[0]

26386

### Check how many duplicates are in this dataset

In [28]:
# Check for duplicates by id 
# df[df.duplicated(subset=['id'], keep=False)].sort_values('id')
df_concat[df_concat.duplicated(subset=['id'], keep=False)].sort_values('id').shape[0] 

273

In [29]:
# Check one id with duplicated rows
df_concat.query('id==202')

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
162,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
163,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
164,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
165,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


When subsetting by id, it results in 273 fields duplicates. The above example shows that apprently similar messages by the same id can be categoried differently. As it is not obvious which categories are correct, we leave similar comments if they are categoried differently. 

Therefore, drop duplicates by all columns (without subset) and leave first only.

In [30]:
df_concat[df_concat.duplicated()].shape[0] # 171 duplicates to remove

171

### Drop the duplicates

In [31]:
df_clean = df_concat.drop_duplicates()

In [32]:
df_clean.shape[0] # from original 26386 records

26215

### Confirm duplicates were removed.

In [33]:
# Confirm duplicates -- should be 0
assert df_clean.duplicated().sum() == 0

### Print the shape of the final dataframe

In [34]:
print(df_clean.shape)

(26215, 40)


## 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [35]:
engine = create_engine('sqlite:///data/DisasterResponse.db')
df_clean.to_sql('messages', engine, index=False, if_exists='replace') 

## 8. Use this notebook to complete `etl_pipeline.py`
Write a script that runs the steps above to create a database based on new datasets specified by the user. 